<strong><b><font size="5">Web Scraping e Análise de Dados</font></b></strong>

<strong><b><font size="4">Extração, Armazenamento e Análise com Web Scraping, Banco de Dados e Linguagem SQL</font></b></strong>

## Fonte de Dados

https://www.worldwildlife.org/species/directory

## Carregando os Pacotes Usados Neste Jupyter Notebook

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.13


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook, se necessário.

In [3]:
# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [4]:
# Imports
import sqlite3 #fornece interface SQL.
import requests #enviar requisições HTTP.
import lxml.html #fornece uma interface para processamento de documentos XML e HTML.
import numpy as np #realizar cálculos numéricos e manipulação de arrays multidimensionais.
import pandas as pd #

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Filipe Fúculo" --iversions

Author: Filipe Fúculo

numpy   : 1.21.5
pandas  : 1.4.4
sqlite3 : 2.6.0
requests: 2.28.1
lxml    : 4.9.1



## Web Scraping

In [6]:
# Cria variável para requisição à página
request_web = requests.get("https://www.worldwildlife.org/species/directory")

In [7]:
# Visualiza a resposta (200 indica resposta de conexão Ok)
request_web

<Response [200]>

In [8]:
type(request_web)

requests.models.Response

In [9]:
# Vamos extrair o código HTML da página e converter em xml para facilitar a manipulação e formatação
#"lxml.html.fromstring" cria um objeto elemento a partir de uma str contendo HTML e cria uma representação em forma de árvore.
html_to_xml = lxml.html.fromstring(request_web.text)

In [10]:
# Visualiza
html_to_xml

<Element html at 0x1c64afad680>

O site tem uma tabela, com **< thead >** e **< tbody >**, ambos com **< tr >**, e esse código extrairá todas as linhas da tabela de uma só vez:

In [11]:
# Extrai todas as linhas
trow = html_to_xml.xpath('//tr') 

In [12]:
# Tipo do elemento
trow[:1]

[<Element tr at 0x1c64af91a90>]

Verificamos se todas as linhas têm o mesmo comprimento, se sim, são da mesma tabela.

In [13]:
# Verifica o comprimento das linhas
[len(T) for T in trow[:10]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

É melhor extrair a tabela usando a tag **< tbody >**, em vez da tag **< tr >**, pois assim extraímos a tabela formatada.

In [14]:
# Extrai os dados da tabela 
tr = html_to_xml.xpath('//tbody') 

In [15]:
# Tipo do elemento
tr

[<Element tbody at 0x1c64afb5b80>]

In [16]:
# Verificamos quantos itens temos na tabela
len(tr[0].getchildren())

50

In [17]:
# Este código faz a mesma coisa que o anterior
tablebody = tr[0]
len(tablebody)

50

In [18]:
# Agora vamos obter o texto de 1 item da tabela
tablebody.getchildren()[0].text_content()

'\nAfrican Elephant\nLoxodonta africana\n\n'

Temos 3 termos (3 colunas). Podemos então usar o código abaixo para retornar cada coluna de uma linha da tabela, como exemplo.

In [19]:
# Extrai colunas de uma linha
tablebody.getchildren()[0].getchildren()

[<Element td at 0x1c64afc5db0>,
 <Element td at 0x1c64afc5680>,
 <Element td at 0x1c64afc5ea0>]

In [20]:
# Um print para cada item
print(tablebody.getchildren()[0].getchildren()[0].text_content())
print(tablebody.getchildren()[0].getchildren()[1].text_content())
print(tablebody.getchildren()[0].getchildren()[2].text_content())

African Elephant
Loxodonta africana



Vamos extrair uma amostra dos dados.

In [21]:
# Dicionário para amostra de dados
data_sample = {"Common name"         :[], 
               "Scientific name"     :[],
               "Conservation status" :[]}

In [22]:
# Loop pela tabela para extrair uma amostra de dados
for row in tablebody.getchildren()[0:3]:
    data_sample["Common name"].append(row.getchildren()[0].text_content())
    data_sample["Scientific name"].append(row.getchildren()[1].text_content())
    data_sample["Conservation status"].append(row.getchildren()[2].text_content())

In [23]:
# Visualiza amostra de dados
data_sample

{'Common name': ['African Elephant',
  'African forest elephant',
  'African savanna elephant'],
 'Scientific name': ['Loxodonta africana', '', 'Loxodonta africana africana'],
 'Conservation status': ['', 'Critically Endangered', 'Endangered']}

In [24]:
# Convertemos a amostra de dados (dicionário) para dataframe do Pandas
pd.DataFrame(data_sample)

,Common name,Scientific name,Conservation status
0,African Elephant,Loxodonta africana,
1,African forest elephant,,Critically Endangered
2,African savanna elephant,Loxodonta africana africana,Endangered


Agora armazenamos todos os dados no dicionário Python.

In [25]:
# Dicionário para receber todos os dados
data = {"Common name"         :[],
        "Scientific name"     :[],
        "Conservation status" :[]}

In [26]:
# Loop para extrair todos os dados da tabela
for row in tablebody.getchildren():
    data["Common name"].append(row.getchildren()[0].text_content().strip())
    data["Scientific name"].append(row.getchildren()[1].text_content().strip())
    data["Conservation status"].append(row.getchildren()[2].text_content().strip())      

In [27]:
# Converte para dataframe do pandas e visualiza
pd.DataFrame(data).head(10)

,Common name,Scientific name,Conservation status
0,African Elephant,Loxodonta africana,
1,African forest elephant,,Critically Endangered
2,African savanna elephant,Loxodonta africana africana,Endangered
3,African Wild Dog,Lycaon pictus,Endangered
4,Albacore Tuna,Thunnus alalunga,Near Threatened
5,Amazon River Dolphin,Scientific Name Inia geoffrensis,
6,Amur Leopard,Panthera pardus orientalis,Critically Endangered
7,Arctic Fox,Vulpes lagopus,Least Concern
8,Arctic Wolf,Canis lupus arctos,Least Concern
9,Asian Elephant,Elephas maximus indicus,Endangered


In [28]:
# Criamos o dataframe
df = pd.DataFrame(data)   

In [29]:
# Substituímos valores ausentes por NaN
# (inplace = True) indica que a substituição deve ser feita diretamente no DataFrame df, em vez de retornar um novo DataFrame com as substituições.
df.replace('', np.nan, inplace = True)

In [30]:
# Iteração pelos itens e impressão na tela
for i in df.itertuples():
    print(i[1:])

('African Elephant', 'Loxodonta africana', nan)
('African forest elephant', nan, 'Critically Endangered')
('African savanna elephant', 'Loxodonta africana africana', 'Endangered')
('African Wild Dog', 'Lycaon pictus', 'Endangered')
('Albacore Tuna', 'Thunnus alalunga', 'Near Threatened')
('Amazon River Dolphin', 'Scientific Name Inia geoffrensis', nan)
('Amur Leopard', 'Panthera pardus orientalis', 'Critically Endangered')
('Arctic Fox', 'Vulpes lagopus', 'Least Concern')
('Arctic Wolf', 'Canis lupus arctos', 'Least Concern')
('Asian Elephant', 'Elephas maximus indicus', 'Endangered')
('Beluga', 'Delphinapterus leucas', 'Near Threatened')
('Bigeye Tuna', 'Thunnus obesus', 'Vulnerable')
('Black Rhino', 'Diceros bicornis', 'Critically Endangered')
('Black Spider Monkey', 'Ateles paniscus', 'Vulnerable')
('Black-footed Ferret', 'Mustela nigripes', 'Endangered')
('Blue Whale', 'Balaenoptera musculus', 'Endangered')
('Bluefin Tuna', 'Thunnus Thynnus', 'Endangered')
('Bonobo', 'Pan paniscus'

## Armazenamento em Banco de Dados e Linguagem SQL

Para usar o módulo sqlite3, primeiramente, devemos criar um objeto de conexão que represente o banco de dados e, opcionalmente, criar um objeto cursor, que o ajudará na execução de todas as instruções SQL.

Mais detalhes em: https://www.sqlite.org/index.html

In [31]:
# Cria conexão ao banco de dados sqlite que será criado somente na memória (não salva o banco de dados em disco)
conn = sqlite3.connect(":memory:")

In [32]:
# Cria o cursor
# Cursor é um objeto que permite percorrer e manipular um conjunto de resultados obtidos de uma consulta em um banco de dados.
cursor = conn.cursor()

In [33]:
# Cria a instrução SQL para criação de tabela no banco de dados
sql1 = """ CREATE TABLE SPECIES(Common_name PRIMARY KEY NOT NULL, Scientific_name, Conservation_status) """

Usamos NOT NULL para as restrições de não permitir NULL nos registros da coluna Common_name. Uma coluna de chave primária não pode ter valores NULL. A palavra-chave PRIMARY em um banco de dados relacional indica que cada linha na coluna é exclusiva.

In [34]:
# Executa a instrução SQL
cursor.execute(sql1)

In [35]:
# Loop pelos dados extraídos da web e inserção na tabela do banco de dados
for rec in df.itertuples():
    
    # Instrução SQL de Insert
    insert_table = """ INSERT INTO SPECIES(Common_name, Scientific_name, Conservation_status) VALUES (?,?,?) """
    
    # Executa a instução
    cursor.execute(insert_table, rec[1:5])

In [36]:
# O commit grava os dados no banco de dados 
conn.commit()

Vamos consultar os dados armazenados na tabela em memória.

In [37]:
# Cria query para selecionar os dados na tabela do banco de dados
# Aspas duplas triplas pode definir uma string que abrange várias linhas sem a necessidade de escapar caracteres especiais ou usar a concatenação de strings.
query1 = """ SELECT * FROM SPECIES """

In [38]:
# Executa a query
resultado1 = cursor.execute(query1)

In [39]:
# Loop pelo retorno da query e impressão dos dados
for row in resultado1:
    print("Common_name = ", row[0])
    print("Scientific_name = ", row[1])
    print("Conservation_status = ", row[2],"\n")
conn.commit()

Common_name =  African Elephant
Scientific_name =  Loxodonta africana
Conservation_status =  None 

Common_name =  African forest elephant
Scientific_name =  None
Conservation_status =  Critically Endangered 

Common_name =  African savanna elephant
Scientific_name =  Loxodonta africana africana
Conservation_status =  Endangered 

Common_name =  African Wild Dog
Scientific_name =  Lycaon pictus
Conservation_status =  Endangered 

Common_name =  Albacore Tuna
Scientific_name =  Thunnus alalunga
Conservation_status =  Near Threatened 

Common_name =  Amazon River Dolphin
Scientific_name =  Scientific Name Inia geoffrensis
Conservation_status =  None 

Common_name =  Amur Leopard
Scientific_name =  Panthera pardus orientalis
Conservation_status =  Critically Endangered 

Common_name =  Arctic Fox
Scientific_name =  Vulpes lagopus
Conservation_status =  Least Concern 

Common_name =  Arctic Wolf
Scientific_name =  Canis lupus arctos
Conservation_status =  Least Concern 

Common_name =  Asi

Vamos agora retornar dados específicos.

In [40]:
# Cria query para selecionar os dados na tabela do banco de dados
# Atentar quanto ao uso das aspas
# Least Conern = Menor Preocupação
query2 = """ SELECT * FROM SPECIES WHERE Conservation_status = "Least Concern" """

In [41]:
# Executa a query
resultado2 = cursor.execute(query2)

In [42]:
# Loop pelo retorno da query e impressão dos dados
for row in resultado2:
    print("Common_name = ", row[0])
    print("Scientific_name = ", row[1])
    print("Conservation_status = ", row[2],"\n")
conn.commit()

Common_name =  Arctic Fox
Scientific_name =  Vulpes lagopus
Conservation_status =  Least Concern 

Common_name =  Arctic Wolf
Scientific_name =  Canis lupus arctos
Conservation_status =  Least Concern 

Common_name =  Bowhead Whale
Scientific_name =  Balaena mysticetus
Conservation_status =  Least Concern 

Common_name =  Brown Bear
Scientific_name =  Ursus arctos
Conservation_status =  Least Concern 

Common_name =  Common Bottlenose Dolphin
Scientific_name =  Tursiops truncates
Conservation_status =  Least Concern 

Common_name =  Gray Whale
Scientific_name =  Eschrichtius robustus
Conservation_status =  Least Concern 



Atualizando valores de tabela no banco de dados usando Chave Primária como referência.

Instruções Update, Insert e Delete são instruções DML (Data Manipulation Language) em Linguagem SQL.

In [43]:
# Cria query para atualizar os dados na tabela do banco de dados
# Atentar quanto ao uso das aspas
dml_1 = """ UPDATE SPECIES SET Conservation_status = "Least-Concern" WHERE Common_name = "Gray Whale" """

In [44]:
# Executa a query
cursor.execute(dml_1)
conn.commit()

In [45]:
# Cria query para selecionar os dados na tabela do banco de dados
# Atentar quanto ao uso das aspas
query3 = """ SELECT * FROM SPECIES WHERE Conservation_status = "Least-Concern" """

In [46]:
# Executa a query
resultado3 = cursor.execute(query3)

In [47]:
# Loop pelo retorno da query e impressão dos dados
for row in resultado3:
    print("Common_name = ", row[0])
    print("Scientific_name = ", row[1])
    print("Conservation_status = ", row[2],"\n")
conn.commit()

Common_name =  Gray Whale
Scientific_name =  Eschrichtius robustus
Conservation_status =  Least-Concern 



Inserindo dados na tabela no banco de dados.

In [48]:
# Cria query para inserir os dados na tabela do banco de dados
# Atentar quanto ao uso das aspas
dml_2 = " INSERT INTO SPECIES(Common_name, Scientific_name, Conservation_status) \
          VALUES ('Cat', 'Felis catus', 'Important' ) "

In [49]:
# Executa a query
cursor.execute(dml_2)
conn.commit()

In [50]:
# Cria query para selecionar os dados na tabela do banco de dados
# Atentar quanto ao uso das aspas
query4 = """ SELECT * FROM SPECIES WHERE Conservation_status = "Important" """

In [51]:
# Executa a query
resultado4 = cursor.execute(query4)

In [52]:
# Loop pelo retorno da query e impressão dos dados
for row in resultado4:
    print("Common_name = ", row[0])
    print("Scientific_name = ", row[1])
    print("Conservation_status = ", row[2],"\n")
conn.commit()

Common_name =  Cat
Scientific_name =  Felis catus
Conservation_status =  Important 



Excluindo os mesmos dados inseridos acima.

In [53]:
# Cria query para deletar os dados na tabela do banco de dados
# Fique atento ao uso das aspas
dml_3 = """ DELETE from SPECIES where Common_name = "Cat" """

In [54]:
# Executa a query
cursor.execute(dml_3)
conn.commit()

In [55]:
# Cria query para selecionar os dados na tabela do banco de dados
# Atentar quanto ao uso das aspas
query5 = """ SELECT * FROM SPECIES WHERE Common_name = "Cat" """

In [56]:
# Executa a query
resultado5 = cursor.execute(query5)

In [57]:
# Loop pelo retorno da query e impressão dos dados
for row in resultado5:
    print("Common_name = ", row[0])
    print("Scientific_name = ", row[1])
    print("Conservation_status = ", row[2],"\n")
conn.commit()

A base de dados não retorna nenhum valor, pois não possui valores que solicitamos.

In [58]:
# Fechamos a conexão com o banco de dados
conn.close()

# Fim